In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import src
import glob

ModuleNotFoundError: No module named 'src'

In [ ]:
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis'
data_dir = f'{top_dir}/data'
out_dir = f'{top_dir}/results'
figure_dir = f'{top_dir}/figures'

In [ ]:
# Get the feature names for the annotated model
feature_categories = ['visual', 'visual', 'visual', 'visual', 'visual',
             'social detection', 'social detection', 
             'social evaluation', 'social evaluation', 'social evaluation', 'social evaluation', 'social evaluation']
features = pd.read_csv(f'{data_dir}/annotations/annotations.csv').columns.to_list()
features.remove('video_name')

In [ ]:
n_subjs = 4
n_features = len(features)

In [ ]:
for sid_ in range(1):
    sid = str(sid_+1).zfill(2)
    yhat = np.load(f'{out_dir}/voxelwise_encoding/sub-{sid}_yhat.npy')
    y = np.load(f'{out_dir}/voxelwise_encoding/sub-{sid}_y.npy')

In [ ]:
for roi in roi_names:
    cur = df.loc[df.ROIs == roi]
    colors = ['#48D4E1', '#8C55FD', '#FADC00']
    cmap = sns.color_palette(colors, as_cmap=True)
    sns.set(style='whitegrid', context='talk', rc={'figure.figsize':(6,5)})
    fig, ax = plt.subplots()
    sns.barplot(x='Features', y='Pearson r',
                data=cur, ax=ax,
                hue='Feature category',
                palette=cmap,
                dodge=False, ci=None)
    g = sns.swarmplot(x='Features', y='Pearson r',
                    data=cur, ax=ax,
                    color='black')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='center')
    ax.get_legend().remove()
    plt.title(roi)
    sns.despine(left=True)
    ax.set_xlabel('')
    ax.set_ylabel('Prediction accuracy ($\it{r}$)')
    plt.tight_layout()
    plt.savefig(f'{figure_dir}/plot_encoding/{roi}.pdf')